In [1]:
import copy
import datetime as dt
import importlib # needed so that we can reload packages
import matplotlib.pyplot as plt
import os, os.path
import numpy as np
import pandas as pd
import pathlib
import sys
import time
from typing import Union
import warnings
warnings.filterwarnings("ignore")

In [2]:
##  IMPORT SISEPUEDE EXAMPLES AND TRANSFORMERS

from sisepuede.manager.sisepuede_examples import SISEPUEDEExamples
from sisepuede.manager.sisepuede_file_structure import SISEPUEDEFileStructure
import sisepuede.core.support_classes as sc
import sisepuede.transformers as trf
import sisepuede.utilities._plotting as spu
import sisepuede.utilities._toolbox as sf


# Use os.getcwd() instead of os.path.dirname(__file__) in a Jupyter notebook
sys.path.append(os.path.join('../', 'utils'))

from TransformationUtils import ExcelYAMLHandler
from TransformationUtils import StrategyCSVHandler


In [3]:
##  SETUP DIRECTORIES

dir_cur = pathlib.Path(os.getcwd())
dir_data = dir_cur.joinpath("data")
dir_transformations = dir_cur.joinpath("transformations")


##  SETUP FILES

fp_data_base = dir_data.joinpath("sisepuede_inputs_uganda.csv")


##  SETUP SOME SISEPUEDE STUFF

file_struct = SISEPUEDEFileStructure()

matt = file_struct.model_attributes
regions = sc.Regions(matt)
time_periods = sc.TimePeriods(matt)

In [4]:
##  BUILD BASE INPUTS

df_inputs_raw = pd.read_csv(fp_data_base)

# pull example data to fill in gaps
examples = SISEPUEDEExamples()
df_inputs_example = examples.input_data_frame

In [5]:
df_inputs_raw.head()

,region,iso_code3,period,pij_lndu_forests_primary_to_croplands,pij_lndu_forests_mangroves_to_wetlands,yf_agrc_nuts_tonne_ha,ef_lndu_conv_forests_secondary_to_other_gg_co2_ha,ef_lndu_conv_forests_secondary_to_wetlands_gg_co2_ha,ef_lndu_conv_other_to_forests_primary_gg_co2_ha,pij_lndu_forests_mangroves_to_forests_secondary,...,frac_agrc_n_in_above_ground_residue_rice,nemomod_entc_total_annual_min_capacity_pp_oil_gw,avgmass_lvst_animal_buffalo_kg,nemomod_entc_total_annual_min_capacity_pp_biomass_gw,ef_frst_forestmethane_primary_kt_ch4_ha,frac_lndu_increasing_net_imports_met_croplands,elasticity_agrc_herbs_and_other_perennial_crops_demand_to_income,nemomod_entc_renewable_energy_tag_pp_oil,frac_trww_n_removed_treated_secondary_anaerobic,energydensity_enfu_mj_per_litre_fuel_gasoline
0,uganda,UGA,0,0.009591,0,0.6913,0.083964,0.087912,0,0,...,0.007,-999,315,-999,0.000001,1,0.2,0,0.4,35
1,uganda,UGA,1,0.011771,0,0.6497,0.083964,0.087912,0,0,...,0.007,-999,315,-999,0.000001,1,0.2,0,0.4,35
2,uganda,UGA,2,0.011129,0,0.6995,0.083964,0.087912,0,0,...,0.007,-999,315,-999,0.000001,1,0.2,0,0.4,35
3,uganda,UGA,3,0.011167,0,0.6757,0.083964,0.087912,0,0,...,0.007,-999,315,-999,0.000001,1,0.2,0,0.4,35
4,uganda,UGA,4,0.011205,0,0.9438,0.083964,0.087912,0,0,...,0.007,-999,315,-999,0.000001,1,0.2,0,0.4,35


In [6]:
df_inputs_example.head()

,region,time_period,avgload_trns_freight_tonne_per_vehicle_aviation,avgload_trns_freight_tonne_per_vehicle_rail_freight,avgload_trns_freight_tonne_per_vehicle_road_heavy_freight,avgload_trns_freight_tonne_per_vehicle_water_borne,avgmass_lvst_animal_buffalo_kg,avgmass_lvst_animal_cattle_dairy_kg,avgmass_lvst_animal_cattle_nondairy_kg,avgmass_lvst_animal_chickens_kg,...,ef_ippu_tonne_cf4_per_tonne_production_electronics,ef_ippu_tonne_cf4_per_tonne_production_metals,ef_ippu_tonne_c3f8_per_tonne_production_chemicals,ef_ippu_tonne_c3f8_per_tonne_production_electronics,ef_ippu_tonne_c4f10_per_mmm_gdp_product_use_ods_other,ef_ippu_tonne_c4f10_per_tonne_production_chemicals,ef_ippu_tonne_c6f14_per_mmm_gdp_product_use_ods_other,ef_ippu_tonne_c6f14_per_tonne_production_chemicals,ef_ippu_tonne_cc4f8_per_tonne_production_chemicals,ef_ippu_tonne_cc4f8_per_tonne_production_electronics
0,costa_rica,0,70.0,2923.0,31.751466,6468.0,322.900664,520.741388,310.599686,1.12759,...,0.000002,4.204879e-07,0.0,1.515048e-07,0.0,0.0,0.0,0.0,0.0,5.669821e-08
1,costa_rica,1,70.0,2923.0,31.751466,6468.0,322.900664,520.741388,310.599686,1.12759,...,0.000002,2.134675e-07,0.0,1.313925e-07,0.0,0.0,0.0,0.0,0.0,5.185820e-08
2,costa_rica,2,70.0,2923.0,31.751466,6468.0,322.900664,520.741388,310.599686,1.12759,...,0.000002,1.821067e-07,0.0,1.060066e-07,0.0,0.0,0.0,0.0,0.0,5.269348e-08
3,costa_rica,3,70.0,2923.0,31.751466,6468.0,322.900664,520.741388,310.599686,1.12759,...,0.000002,2.094712e-07,0.0,1.093024e-07,0.0,0.0,0.0,0.0,0.0,5.500450e-08
4,costa_rica,4,70.0,2923.0,31.751466,6468.0,322.900664,520.741388,310.599686,1.12759,...,0.000002,3.446161e-07,0.0,1.046451e-07,0.0,0.0,0.0,0.0,0.0,5.266081e-08


In [7]:
def compare_dfs(df1, df2):
    # Assuming your DataFrames are df1 and df2
    columns_df1 = set(df1.columns)
    columns_df2 = set(df2.columns)

    # Columns present in df1 but not in df2
    diff_in_df1 = columns_df1 - columns_df2

    # Columns present in df2 but not in df1
    diff_in_df2 = columns_df2 - columns_df1

    print("Columns in df1 but not in df2:", diff_in_df1)
    print("Columns in df2 but not in df1:", diff_in_df2)

def sync_columns(df1, df2):
    """
    Synchronize columns between df1 and df2:
    - Add columns from df1 to df2 (if missing).
    - Remove columns from df2 that are not in df1.
    """
    # Identify columns to add to df2
    columns_to_add = [col for col in df1.columns if col not in df2.columns]
    
    # Add missing columns to df2 with default values (e.g., NaN)
    for col in columns_to_add:
        df2[col] = df1[col]

    # Identify columns to remove from df2
    columns_to_remove = [col for col in df2.columns if col not in df1.columns]
    
    # Drop unnecessary columns from df2
    df2.drop(columns=columns_to_remove, inplace=True)
    
    return df2


In [8]:
compare_dfs(df_inputs_example, df_inputs_raw)

Columns in df1 but not in df2: {'ef_trns_mobile_combustion_water_borne_kg_n2o_per_tj_ammonia', 'frac_trns_fuelmix_water_borne_ammonia', 'ef_enfu_stationary_combustion_tonne_n2o_per_tj_fuel_ammonia', 'strategy_id', 'energydensity_gravimetric_enfu_gj_per_tonne_fuel_ammonia', 'nemomod_entc_output_activity_ratio_fuel_production_fp_hydrogen_reformation_ccs_hydrogen', 'nemomod_entc_output_activity_ratio_fuel_production_fp_ammonia_production_ammonia', 'cost_enfu_fuel_ammonia_usd_per_tonne', 'nemomod_entc_fixed_cost_fp_ammonia_production_mm_usd_per_gw', 'nemomod_entc_grid_power_constraint_mmm_usd', 'nemomod_entc_input_activity_ratio_fuel_production_fp_ammonia_production_hydrogen', 'energydensity_gravimetric_enfu_gj_per_tonne_fuel_water', 'nemomod_entc_frac_min_share_production_fp_hydrogen_reformation_ccs', 'nemomod_entc_fixed_cost_fp_hydrogen_reformation_ccs_mm_usd_per_gw', 'nemomod_entc_emissions_activity_ratio_fuel_production_fp_hydrogen_reformation_ccs_tonne_co2_per_tj', 'ef_enfu_stationary

In [10]:
df_inputs_raw_complete = sync_columns(df_inputs_example, df_inputs_raw.copy())
df_inputs_raw_complete.head()

,region,pij_lndu_forests_primary_to_croplands,pij_lndu_forests_mangroves_to_wetlands,yf_agrc_nuts_tonne_ha,ef_lndu_conv_forests_secondary_to_other_gg_co2_ha,ef_lndu_conv_forests_secondary_to_wetlands_gg_co2_ha,ef_lndu_conv_other_to_forests_primary_gg_co2_ha,pij_lndu_forests_mangroves_to_forests_secondary,ef_lndu_conv_wetlands_to_settlements_gg_co2_ha,pij_lndu_grasslands_to_forests_secondary,...,nemomod_entc_input_activity_ratio_fuel_production_fp_hydrogen_electrolysis_water,nemomod_entc_input_activity_ratio_fuel_production_fp_hydrogen_reformation_ccs_electricity,energydensity_gravimetric_enfu_gj_per_tonne_fuel_ammonia,energydensity_gravimetric_enfu_gj_per_tonne_fuel_water,frac_trns_fuelmix_water_borne_ammonia,nemomod_entc_output_activity_ratio_fuel_production_fp_ammonia_production_ammonia,nemomod_entc_output_activity_ratio_fuel_production_fp_hydrogen_reformation_ccs_hydrogen,nemomod_entc_frac_min_share_production_fp_hydrogen_reformation_ccs,nemomod_entc_input_activity_ratio_fuel_production_fp_hydrogen_reformation_ccs_natural_gas,nemomod_entc_input_activity_ratio_fuel_production_fp_hydrogen_reformation_ccs_oil
0,uganda,0.009591,0,0.6913,0.083964,0.087912,0,0,0,0,...,0.000004,0,18.6,0.00005,0.0,1,1,0.0,1.315,0.0
1,uganda,0.011771,0,0.6497,0.083964,0.087912,0,0,0,0,...,0.000004,0,18.6,0.00005,0.0,1,1,0.0,1.315,0.0
2,uganda,0.011129,0,0.6995,0.083964,0.087912,0,0,0,0,...,0.000004,0,18.6,0.00005,0.0,1,1,0.0,1.315,0.0
3,uganda,0.011167,0,0.6757,0.083964,0.087912,0,0,0,0,...,0.000004,0,18.6,0.00005,0.0,1,1,0.0,1.315,0.0
4,uganda,0.011205,0,0.9438,0.083964,0.087912,0,0,0,0,...,0.000004,0,18.6,0.00005,0.0,1,1,0.0,1.315,0.0


In [11]:
compare_dfs(df_inputs_example, df_inputs_raw_complete)

Columns in df1 but not in df2: set()
Columns in df2 but not in df1: set()


In [12]:
# Get a list of columns with NaN values
# Get a list of columns with NaN values
columns_with_na = df_inputs_raw_complete.columns[df_inputs_raw_complete.isna().any()].tolist()

print(columns_with_na)


['exports_enfu_pj_fuel_kerosene', 'frac_enfu_fuel_demand_imported_pj_fuel_kerosene', 'frac_enfu_fuel_demand_imported_pj_fuel_oil', 'exports_enfu_pj_fuel_coal', 'frac_enfu_fuel_demand_imported_pj_fuel_electricity', 'exports_enfu_pj_fuel_natural_gas', 'exports_enfu_pj_fuel_gasoline', 'exports_enfu_pj_fuel_oil', 'frac_enfu_fuel_demand_imported_pj_fuel_hydrocarbon_gas_liquids', 'exports_enfu_pj_fuel_electricity', 'exports_enfu_pj_fuel_diesel', 'frac_enfu_fuel_demand_imported_pj_fuel_natural_gas', 'frac_enfu_fuel_demand_imported_pj_fuel_coal', 'frac_enfu_fuel_demand_imported_pj_fuel_crude', 'frac_enfu_fuel_demand_imported_pj_fuel_gasoline', 'exports_enfu_pj_fuel_crude', 'exports_enfu_pj_fuel_hydrocarbon_gas_liquids', 'frac_enfu_fuel_demand_imported_pj_fuel_diesel']


#  Let's try building transformations using this


In [13]:
transformers = trf.transformers.Transformers(
    {},
    df_input = df_inputs_raw_complete,
)


##  Instantiate some transformations CAREFUL NOT TO OVERWRITE EXISTING TRANSFORMATIONS!!

In [14]:
# set an ouput path and instantiate
if not dir_transformations.exists():
    trf.instantiate_default_strategy_directory(
        transformers,
        dir_transformations,
    )

##  --HERE, CUSTOMIZE YOUR TRANSFORMATIONS AND STRATEGIES--
- go to `dir_transformers` and edit config files and `strategy_definitions.csv`
- then, go to next cell

In [15]:
excel_yaml_handler = ExcelYAMLHandler(excel_file='data/ssp_uganda_transformation_cw_vf.xlsx', yaml_directory='transformations')

In [16]:
excel_yaml_handler.process_yaml_files()

YAML file transformation_agrc_inc_conservation_agriculture.yaml for strategy strategy_priority_mitigation set to default because it does not have magnitude attribute
YAML file transformation_agrc_inc_conservation_agriculture.yaml for strategy strategy_aditional_actions set to default because it does not have magnitude attribute
YAML file transformation_agrc_inc_conservation_agriculture.yaml for strategy strategy_net_zero set to default because it does not have magnitude attribute
YAML file transformation_inen_shift_fuel_heat.yaml for strategy strategy_net_zero set to default because it does not have magnitude attribute
YAML file transformation_lsmm_inc_management_cattle_pigs.yaml for strategy strategy_net_zero set to default because it does not have magnitude attribute
YAML file transformation_lsmm_inc_management_other.yaml for strategy strategy_net_zero set to default because it does not have magnitude attribute
YAML file transformation_lsmm_inc_management_poultry.yaml for strategy st

In [18]:
csv_handler = StrategyCSVHandler('transformations/strategy_definitions.csv', 'transformations', '../utils/strategy_mapping.yaml')

csv_handler.add_row('PFLO', 'NDC priority', 'strategy_priority_mitigation')
csv_handler.add_row('PFLO', 'NDC aditional actions', 'strategy_aditional_actions')
csv_handler.add_row('PFLO', 'Net Zero', 'strategy_net_zero')

csv_handler.save_csv()

Error: strategy_code PFLO:STRATEGY_PRIORITY_MITIGATION already exists. Please use a different code or modify the existing one.
Error: strategy_code PFLO:STRATEGY_ADITIONAL_ACTIONS already exists. Please use a different code or modify the existing one.
Error: strategy_code PFLO:STRATEGY_NET_ZERO already exists. Please use a different code or modify the existing one.
Data saved to transformations/strategy_definitions.csv


In [17]:
# import yaml

# # Path to the YAML file
# file_path = dir_transformations.joinpath("transformation_pflo_inc_ind_ccs_strategy_net_zero.yaml")

# # Read the YAML file
# with open(file_path, 'r') as file:
#     config = yaml.safe_load(file)

# # Update the value of 'dict_magnitude_eff'
# config['parameters']['dict_magnitude_eff'] = 0.05
# config['parameters']['dict_magnitude_prev'] = {
#     'cement': 0.5,
#     'chemicals': 0.5,
#     'metals': 0.5,
#     'plastics': 0.5
# }

# # Save the changes back to the YAML file
# with open(file_path, 'w') as file:
#     yaml.safe_dump(config, file)

# print(f"File updated successfully")


In [19]:
# then, you can load this back in after modifying (play around with it)
transformations = trf.Transformations(
    dir_transformations,
    transformers = transformers,
)
tab = transformations.attribute_transformation.table

In [20]:
#  build the strategies -- will export to path
t0 = time.time()
strategies = trf.Strategies(
    transformations,
    export_path = "transformations",
    prebuild = True,
)

t_elapse = sf.get_time_elapsed(t0)
print(f"Strategies defined at {strategies.transformations.dir_init} initialized in {t_elapse} seconds")

Strategies defined at /home/fabian_fuentes/repos/sisepuede_region_nbs_1.1/uganda/transformations initialized in 3.45 seconds


In [21]:
strategies.attribute_table

,strategy_id,strategy_code,strategy,description,transformation_specification,baseline_strategy_id
0,0,BASE,Strategy TX:BASE,NaN,TX:BASE,1
1,1000,AGRC:DEC_CH4_RICE,Singleton - Default Value - AGRC: Improve rice...,NaN,TX:AGRC:DEC_CH4_RICE,0
2,1001,AGRC:DEC_EXPORTS,Singleton - Default Value - AGRC: Decrease Exp...,NaN,TX:AGRC:DEC_EXPORTS,0
3,1002,AGRC:DEC_LOSSES_SUPPLY_CHAIN,Singleton - Default Value - AGRC: Reduce suppl...,NaN,TX:AGRC:DEC_LOSSES_SUPPLY_CHAIN,0
4,1003,AGRC:INC_CONSERVATION_AGRICULTURE,Singleton - Default Value - AGRC: Expand conse...,NaN,TX:AGRC:INC_CONSERVATION_AGRICULTURE,0
...,...,...,...,...,...,...
67,6001,PFLO:INC_IND_CCS,Singleton - Default Value - PFLO: Industrial c...,NaN,TX:PFLO:INC_IND_CCS,0
68,6002,PFLO:ALL,All Actions,All actions (unique by transformer),TX:AGRC:DEC_CH4_RICE|TX:AGRC:DEC_EXPORTS|TX:AG...,0
69,6003,PFLO:STRATEGY_PRIORITY_MITIGATION,strategy_priority_mitigation,NDC priority,TX:TRDE:DEC_DEMAND_STRATEGY_PRIORITY_MITIGATIO...,0
70,6004,PFLO:STRATEGY_ADITIONAL_ACTIONS,strategy_aditional_actions,NDC aditional actions,TX:ENTC:TARGET_RENEWABLE_ELEC_STRATEGY_ADITION...,0


##  Build our templates
- let's use the default variable groupings for LHS

In [22]:
df_vargroups = examples("variable_trajectory_group_specification")

strategies.build_strategies_to_templates(
    df_trajgroup = df_vargroups,
    include_simplex_group_as_trajgroup = True,
    strategies = [0, 6003, 6004, 6005],
)

0

# Finally, load SISEPUEDE so that we can run it

In [23]:
import sisepuede as si

ssp = si.SISEPUEDE(
    "calibrated",
    db_type = "csv",
    #id_str = "sisepuede_run_2024-11-06",
    initialize_as_dummy = False, # no connection to Julia is initialized if set to True
    regions = ["uganda"],
    strategies = strategies,
    try_exogenous_xl_types_in_variable_specification = True,
)

2025-01-20 16:24:20,347 - INFO - Successfully initialized SISEPUEDEFileStructure.
2025-01-20 16:24:20,349 - WARNING - Missing key dict_dimensional_keys: key time_series not found. Tables that rely on the time_series will not have index checking.
2025-01-20 16:24:20,349 - INFO - 	Setting export engine to 'csv'.
2025-01-20 16:24:20,350 - WARNING - No index fields defined. Index field values will not be checked when writing to tables.
2025-01-20 16:24:20,350 - INFO - Successfully instantiated table ANALYSIS_METADATA
2025-01-20 16:24:20,351 - WARNING - No index fields found in ATTRIBUTE_DESIGN. Initializing index fields.
2025-01-20 16:24:20,351 - INFO - Successfully instantiated table ATTRIBUTE_DESIGN
2025-01-20 16:24:20,351 - WARNING - No index fields found in ATTRIBUTE_LHC_SAMPLES_EXOGENOUS_UNCERTAINTIES. Initializing index fields.
2025-01-20 16:24:20,351 - INFO - Successfully instantiated table ATTRIBUTE_LHC_SAMPLES_EXOGENOUS_UNCERTAINTIES
2025-01-20 16:24:20,352 - WARNING - No index fi

Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython


Precompiling NemoMod...
Info Given NemoMod was explicitly requested, output will be shown live 
ERROR: Method overwriting is not permitted during Module precompilation. Use `__precompile__(false)` to opt-out of precompilation.
   3037.6 ms  ? NemoMod
[ Info: Precompiling NemoMod [a3c327a0-d2f0-11e8-37fd-d12fd35c3c72] 
ERROR: Method overwriting is not permitted during Module precompilation. Use `__precompile__(false)` to opt-out of precompilation.
┌ Info: Skipping precompilation due to precompilable error. Importing NemoMod [a3c327a0-d2f0-11e8-37fd-d12fd35c3c72].
└   exception = Error when precompiling module, potentially caused by a __precompile__(false) declaration in the module.
2025-01-20 16:25:11,821 - INFO - Successfully initialized JuMP optimizer from solver module HiGHS.
2025-01-20 16:25:11,860 - INFO - Successfully initialized SISEPUEDEModels.
2025-01-20 16:25:11,870 - INFO - Table ANALYSIS_METADATA successfully written to /home/fabian_fuentes/anaconda3/envs/ssp2/lib/python3.11

In [24]:
dict_scens = {
    ssp.key_design: [0],
    ssp.key_future: [0],
    ssp.key_strategy: [0, 6003, 6004, 6005],
}

ssp.project_scenarios(
    dict_scens,
    save_inputs = True,
)

2025-01-20 16:25:14,951 - INFO - 
***	STARTING REGION uganda	***

2025-01-20 16:25:17,837 - INFO - Trying run primary_id = 0 in region uganda
2025-01-20 16:25:17,837 - INFO - Running AFOLU model
2025-01-20 16:25:17,998 - INFO - AFOLU model run successfully completed
2025-01-20 16:25:17,999 - INFO - Running CircularEconomy model
2025-01-20 16:25:18,049 - INFO - CircularEconomy model run successfully completed
2025-01-20 16:25:18,050 - INFO - Running IPPU model
2025-01-20 16:25:18,127 - INFO - IPPU model run successfully completed
2025-01-20 16:25:18,128 - INFO - Running Energy model (EnergyConsumption without Fugitive Emissions)
2025-01-20 16:25:18,158 - DEBUG - Missing elasticity information found in 'project_energy_consumption_by_fuel_from_effvars': using specified future demands.
2025-01-20 16:25:18,206 - ERROR - Error running EnergyConsumption without Fugitive Emissions: Invalid summations found: some categories exceed the sum threshold.Energy fractions by category do not sum to 1. 

{'uganda': [0, 69069, 70070, 71071]}

In [24]:
df_run = ssp.generate_scenario_database_from_primary_key(0)
df_run = df_run.get("georgia")
df_run.to_csv("/home/fabian_fuentes/repos/sisepuede_region_nbs/georgia/output/uganda_baselines_temp.csv", encoding = "UTF-8", index = None, )

AttributeError: 'NoneType' object has no attribute 'to_csv'

In [30]:
df_out = ssp.read_output(None)

In [31]:
df_in = ssp.read_input(None)

In [ ]:
df_out.primary_id.unique()

# option to pass `df_out` back to data_modifications
- do this if the outputs are needed to rescale input demand values to match IEA data

In [33]:
(
    df_out
    .to_csv(
        dir_data.joinpath("sisepuede_outputs_georgia.csv"),
        index = None,
        encoding = "UTF-8",
    )
)

In [ ]:
import matplotlib.pyplot as plt

# Configuración del gráfico
fig, ax = plt.subplots(figsize=(18, 8))
ax.set_xlabel("Time Period")
ax.set_ylabel("MT Emissions CO2e ")

df_plot = df_out[df_out[ssp.key_primary].isin([72072])]

fields = matt.get_all_subsector_emission_total_fields()
dict_format = dict((k, {"color": v}) for (k, v) in matt.get_subsector_color_map().items())

fig, ax = spu.plot_stack(
    df_plot,
    fields,
    dict_formatting=dict_format,
    field_x="time_period",
    figtuple=(fig, ax),
)

handles = [plt.Line2D([0], [0], color=formatting["color"], lw=4) for formatting in dict_format.values()]
labels = list(dict_format.keys())

ax.legend(handles, labels, title="Subsector Emissions", loc="upper left", bbox_to_anchor=(1, 1))

plt.show()


In [ ]:
# Configuración del gráfico
fig, ax = plt.subplots(figsize=(18, 8))
ax.set_xlabel("Time Period")
ax.set_ylabel("MT Emissions CO2e ")

df_plot = df_out[df_out[ssp.key_primary].isin([72072])]

fields = matt.get_all_subsector_emission_total_fields()
dict_format = dict((k, {"color": v}) for (k, v) in matt.get_subsector_color_map().items())

fig, ax = spu.plot_stack(
    df_plot,
    fields,
    dict_formatting=dict_format,
    field_x="time_period",
    figtuple=(fig, ax),
)

handles = [plt.Line2D([0], [0], color=formatting["color"], lw=4) for formatting in dict_format.values()]
labels = list(dict_format.keys())

ax.legend(handles, labels, title="Subsector Emissions", loc="upper left", bbox_to_anchor=(1, 1))

plt.show()


In [ ]:
strategys = df_out.primary_id.unique()
strategys

In [ ]:
# Configuración del gráfico con tres subplots (uno para cada estrategia)
fig, axes = plt.subplots(nrows=1, ncols=5, figsize=(18, 6), sharey=True)
fig.suptitle("Comparación de Emisiones CO2e por Estrategia")

# Iterar sobre cada estrategia y su respectivo eje
for ax, strategy in zip(axes, strategys):
    # Filtrar el DataFrame para la estrategia actual
    df_plot = df_out[df_out[ssp.key_primary].isin([strategy])]
    
    # Obtener los campos y el formato de color para cada subsector
    fields = matt.get_all_subsector_emission_total_fields()
    dict_format = dict((k, {"color": v}) for (k, v) in matt.get_subsector_color_map().items())
    
    # Graficar con el stackplot para la estrategia actual en su propio eje
    spu.plot_stack(
        df_plot,
        fields,
        dict_formatting=dict_format,
        field_x="time_period",
        figtuple=(fig, ax),
    )
    
    # Configurar etiquetas y título del subplot
    ax.set_title(f"Estrategia {strategy}")
    ax.set_xlabel("Time Period")
    ax.set_ylabel("MT Emissions CO2e")

# Mostrar el gráfico completo
plt.tight_layout(rect=[0, 0, 1, 0.95])  # Ajusta para que el título principal no se superponga

handles = [plt.Line2D([0], [0], color=formatting["color"], lw=4) for formatting in dict_format.values()]
labels = list(dict_format.keys())

ax.legend(handles, labels, title="Subsector Emissions", loc="upper left", bbox_to_anchor=(1, 1))

plt.show()



# Export Wide File

In [129]:
df_out = ssp.read_output(None)
df_in = ssp.read_input(None)

In [ ]:
df_out.head()

In [ ]:
df_in.head()

In [ ]:
all_primaries = sorted(list(df_out[ssp.key_primary].unique()))
all_primaries

In [ ]:
# build if unable to simply read the data frame
if df_in is None:
    df_in = []
     
    for region in ssp.regions:
        for primary in all_primaries: 
            df_in_filt = ssp.generate_scenario_database_from_primary_key(primary)
            df_in.append(df_in_filt.get(region))
    
    df_in = pd.concat(df_in, axis = 0).reset_index(drop = True)

df_in 


In [ ]:
df_export = pd.merge(
    df_out,
    df_in,
    how = "left",
)
df_export

In [ ]:
# check output directory 
dir_pkg = os.path.join(
    ssp.file_struct.dir_out, 
    f"sisepuede_summary_results_run_{ssp.id_fs_safe}"
)
os.makedirs(dir_pkg) if not os.path.exists(dir_pkg) else None
dir_pkg

In [136]:
for tab in ["ATTRIBUTE_STRATEGY"]:
    ssp.database.db.read_table(tab).to_csv(
        os.path.join(dir_pkg, f"{tab}.csv"),
        index = None,
        encoding = "UTF-8"
    )


In [137]:
df_primary = (
    ssp
    .odpt_primary
    .get_indexing_dataframe(
        sorted(list(df_out[ssp.key_primary].unique()))
    )
)
    
df_primary.to_csv(
    os.path.join(dir_pkg, f"ATTRIBUTE_PRIMARY.csv"),
    index = None,
    encoding = "UTF-8"
)

df_export.to_csv(
    os.path.join(dir_pkg, f"sisepuede_results_{ssp.id_fs_safe}_WIDE_INPUTS_OUTPUTS.csv"),
    index = None,
    encoding = "UTF-8"
)

In [ ]:
ssp.file_struct.dir_out